<a href="https://colab.research.google.com/github/ddPn08/automatic1111-colab/blob/main/automatic1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Stable Diffusion WebUI Colab](https://github.com/ddPn08/stable-diffusion-webui-colab) by [ddPn08](https://github.com/ddpn08/)

This colab runs from the repo [`automatic1111`](https://github.com/AUTOMATIC1111/stable-diffusion-webui)  
このColabは[`automatic1111`](https://github.com/AUTOMATIC1111/stable-diffusion-webui) を使用しています。


## 1 - Setup

In [ ]:
! nvidia-smi

### 1.1 Download repo and install

Clone git repo and setup miniconda
> Gitリポジトリをクローン、minicondaのセットアップ

In [ ]:
# @markdown ## Clone the automatic1111/stable-diffusion-webui

# Define a global variable
ngrok_proc = None


webui_branch = "master"  # @param {type: "string"}

! git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui
! git checkout {webui_branch}

import sys

! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
! chmod +x Miniconda3-latest-Linux-x86_64.sh
! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append("/usr/local/lib/python3.8/site-packages/")
! rm Miniconda3-latest-Linux-x86_64.sh
! conda env update -n base -f environment-wsl2.yaml

! mkdir repositories
! git clone https://github.com/CompVis/stable-diffusion.git repositories/stable-diffusion
! git clone https://github.com/CompVis/taming-transformers.git repositories/taming-transformers
! git clone https://github.com/sczhou/CodeFormer.git repositories/CodeFormer
! git clone https://github.com/salesforce/BLIP.git repositories/BLIP

! pip install transformers==4.19.2 diffusers invisible-watermark --prefer-binary
! pip install git+https://github.com/crowsonkb/k-diffusion.git --prefer-binary
! pip install git+https://github.com/TencentARC/GFPGAN.git --prefer-binary
! pip install -r repositories/CodeFormer/requirements.txt --prefer-binary
! pip install -r requirements.txt  --prefer-binary
! pip install -U numpy  --prefer-binary

! pip install pyngrok

### 1.2 Setup models

In [ ]:
# @markdown # Load the stable-diffusion model
# @markdown > ### stable-diffusionのモデルをロード

# @markdown **Model Path Variables**
# ask for the link
print("Local Path Variables:\n")

model_filename = "sd-v1-4.ckpt"  # @param {type:"string"}
# @markdown First load the model specified by `model_filename`
# @markdown > `model_filename`で指定したモデルをはじめにロードする
specify_model = True # @param {type:"boolean"}
models_path = "/content/models"  # @param {type:"string"}
output_path = "/content/output"  # @param {type:"string"}
config_path = "/content/config"  # @param {type:"string"}
embeddings_path = "/content/embeddings"  # @param {type:"string"}

# @markdown **Download the model if it isn't already in the `models_path` folder (Optional)**
# @markdown > モデルが`models_path`フォルダーにない場合に、モデルをダウンロードする

# @markdown To download the model, you need to have accepted the terms [HERE](https://huggingface.co/CompVis/stable-diffusion-v1-4)
# @markdown and have copied a token from [HERE](https://huggingface.co/settings/tokens)

# @markdown > モデルをダウンロードするには, [このページ](https://huggingface.co/CompVis/stable-diffusion-v1-4)で条件に同意する必要があります。
# @markdown > また、[このページ](https://huggingface.co/settings/tokens)からトークンを取得する必要があります。
download_if_missing = False  # @param {type:"boolean"}
model_url = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt"  # @param {type:"string"}
token = ""  # @param {type:"string"}

# @markdown **Google Drive Path Variables (Optional)**
mount_google_drive = True  # @param {type:"boolean"}
force_remount = False

%cd /content/
import os

mount_success = True
if mount_google_drive:
    from google.colab import drive

    try:
        drive_path = "/content/drive"
        drive.mount(drive_path, force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models"  # @param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/automatic1111/outputs"  # @param {type:"string"}
        config_path_gdrive = "/content/drive/MyDrive/AI/automatic1111/config"  # @param {type:"string"}
        embeddings_path_gdrive = "/content/drive/MyDrive/AI/textual-inversion/embeddings"  # @param {type:"string"}
        models_path = models_path_gdrive
        output_path = output_path_gdrive
        config_path = config_path_gdrive
        embeddings_path = embeddings_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")
        mount_success = False

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(embeddings_path, exist_ok=True)

if not os.path.exists(f"{config_path}/config.json"):
    with open(f"{config_path}/config.json", mode="w") as f:
        f.write('{"outdir_samples": "' + output_path + '"}')

if download_if_missing:
    if not mount_success:
        print("Downloading model to " + models_path + " due to gdrive mount error")
    elif not os.path.exists(models_path + "/" + model_filename):
        ! mkdir sd-model
        %cd /content/sd-model/
        ! curl -LJ  {model_url} -o {model_filename} {'-H "Authorization: Bearer ' + token + '"' if token else ""}
        ! mv /content/sd-model/{model_filename} {models_path}/
        %cd /content/
    else:
        print("Model already downloaded, moving to next step")

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")
print(f"config_path: {config_path}")
print(f"embeddings_path: {embeddings_path}")

In [ ]:
# @markdown ## Download GFPGAN model
# @markdown **GFPGAN** Automatically correct distorted faces with a built-in GFPGAN option, fixes them in less than half a second
# @markdown > **GFPGAN** オプションで歪んだ顔を自動的に修正し、0.5秒未満で修正します

%cd stable-diffusion-webui
! curl -LOJ https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth

## 2 Optional - Set webUI settings and configs before running

In [ ]:
# @markdown # Launch preferences - Advanced
# @markdown # 詳細設定

# @markdown Click here for more information -> https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Optimizations
# @markdown > 詳しい情報はこちらから -> https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Optimizations

no_half = False  # @param {type:"boolean"}
# @markdown * Do not switch the model to 16-bit floats
# @markdown > モデルを16ビットfloatに切り替えない
no_progressbar_hiding = False  # @param {type:"boolean"}
# @markdown * Do not hide progressbar in gradio UI (we hide it because it slows down ML if you have hardware acceleration in browser)
# @markdown > プログレスバーを非表示にしない (ブラウザにハードウェアアクセラレーションが場合MLが遅くなるため非表示にすることを推奨します)
medvram = False  # @param {type:"boolean"}
# @markdown * Makes the Stable Diffusion model consume less VRAM by splitting it into three parts - cond (for transforming text into numerical representation), first_stage (for converting a picture into latent space and back), and unet (for actual denoising of latent space) and making it so that only one is in VRAM at all times, sending others to CPU RAM. Lowers performance, but only by a bit - except if live previews are enabled.
# @markdown > Stable Diffusion モデルが VRAM の消費を少なくするために、cond(テキストを数値表現に変換するため)、first_stage(画像を潜在空間に変換して元に戻すため)、およびunet(潜在空間の実際のノイズ除去のため) の3つの部分に分割し、 常に1つだけがVRAMにあり、残りはCPU RAMに送信されるようにします。 (パフォーマンスが少し低下します。ライブプレビューが有効になっている場合さらに性能低下が大きくなります。)
lowvram = False  # @param {type:"boolean"}
# @markdown * An even more thorough optimization of the above, splitting unet into many modules, and only one module is kept in VRAM. Devastating for performance.
# @markdown > `medvram`をさらに徹底的に最適化し、unetを多くのモジュールに分割し、1つのモジュールのみをVRAMに保持します。 パフォーマンスは壊滅的。
disable_opt_split_attention = False  # @param {type:"boolean"}
# @markdown * Force-disables cross-attention layer optimization.
# @markdown > クロスアテンションレイヤーの最適化を強制的に無効にします。

run_string_with_variables = {
    "--no-half": f"{no_half}",
    "--no-progressbar-hiding": f"{no_progressbar_hiding}",
    "--medvram": f"{medvram}",
    "--lowvram": f"{lowvram}",
    "--disable-opt-split-attention": f"{disable_opt_split_attention}",
}

only_true_vars = {k for (k, v) in run_string_with_variables.items() if v == "True"}

## 3 - Launch WebUI for stable diffusion

In [ ]:
# @markdown # Start Ngrok Tunnel (Optional)
# @markdown > Ngrokのトンネルを起動する (オプション)

# @markdown Use Ngrok tunneling for more stable communication. To use this, you need to [sign up](https://ngrok.com/) for Ngrok and [get a token](https://dashboard.ngrok.com/get-started/setup).
# @markdown > より安定した通信のために Ngrok トンネリングを使用します。 これを使用するには、Ngrok に[サインアップ]((https://ngrok.com/) して[トークンを取得](https://dashboard.ngrok.com/get-started/setup) する必要があります。

use_ngrok = False  # @param {type: "boolean"}
ngrok_auth_token = ""  # @param {type: "string"}
ngrok_region = "us"  # @param ["us", "eu", "au", "ap", "sa", "jp", "in"]
port = 7860  # @param {type: "number"}

if use_ngrok:
    from pyngrok import ngrok

    if ngrok_proc:
        ngrok_proc.proc.kill()
    pyngrok_config = ngrok.conf.PyngrokConfig(
        auth_token=ngrok_auth_token, region=ngrok_region
    )
    tunnel = ngrok.connect(f"localhost:{port}", pyngrok_config=pyngrok_config)
    ngrok_proc = ngrok.get_ngrok_process()
    print(tunnel)

In [ ]:
#@markdown keep in mind that this script is set to run for ever.
#@markdown > ※注意 このスクリプトは永久に実行されます。

#@markdown # Important - click the public URL to launch WebUI in another tab
#@markdown > ### 重要 - 公開URLをクリックしてWebUIを起動してください

#@markdown ![](https://user-images.githubusercontent.com/71378929/189563599-6df78bcf-133b-41e8-a55d-8ca3783cd933.png)

import os

transformers_offline = False # @param {type: "boolean"}
os.environ['TRANSFORMERS_OFFLINE'] = "1" if transformers_offline else "0"

vars = " ".join(only_true_vars)
if not ngrok_proc:
  vars += " --share"

if specify_model:
  vars += f" --ckpt {models_path}/{model_filename}"

! git pull
! python webui.py \
  --ckpt-dir {models_path} \
  --ui-config-file {config_path}/ui-config.json \
  --ui-settings-file {config_path}/config.json \
  --styles-file {config_path}/styles.csv \
  $vars